In [2]:
import os
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm


sem_path = "/home/rozenberszki/Downloads/perf_render"
rgb_psum = np.zeros((3), dtype=np.longlong)
rgb_psum_sq = np.zeros((3), dtype=np.longlong)

sh_psum = np.zeros((27), dtype=np.float64)
sh_psum_sq = np.zeros((27), dtype=np.float64)

pixel_count = 0
im_size=124*124

In [3]:
for scene_dir in tqdm(sorted(os.listdir(sem_path))):
    scene_path = os.path.join(sem_path, scene_dir)
    for image in sorted(os.listdir(scene_path)):
        file_path = os.path.join(scene_path,image)
        if(image.endswith(".jpg")):
            im_path = os.path.join(scene_path,file_path)
            pil_im = Image.open(im_path)
            im_array = np.array(pil_im, dtype=np.longlong)
            rgb_psum += im_array.sum(axis=(0,1))
            rgb_psum_sq += (im_array**2).sum(axis=(0,1))
            pixel_count+=im_size
        elif(image.endswith(".npz")):
            sh_path = os.path.join(scene_path,file_path)
            sh_loaded = np.load(sh_path)['arr_0']
            sh_psum += sh_loaded.sum(axis=(0,1))
            sh_psum_sq += (sh_loaded**2).sum(axis=(0,1))

  0%|          | 0/1513 [00:00<?, ?it/s]

In [4]:
print(rgb_psum)
print(rgb_psum_sq)
print(sh_psum)
print(sh_psum_sq)

[240330371983 216180015742 187586618193]
[40616836773107 34405259425052 27961185800763]
[ 1.54504007e+07  2.16313196e+08  1.14159922e+08  1.15397989e+08
  1.09199390e+08  1.11284946e+08  1.71871185e+08  1.11106533e+08
  2.11451451e+08 -7.51691670e+07  2.63574392e+08  1.16203848e+08
  1.12332876e+08  1.08533821e+08  1.10072505e+08  1.54339430e+08
  1.11520222e+08  1.72693571e+08 -1.60399273e+08  3.15005503e+08
  1.17611523e+08  1.08952170e+08  1.09273393e+08  1.07585975e+08
  1.22794102e+08  1.12044101e+08  1.15787992e+08]
[2.50887713e+08 2.76291228e+08 1.94118073e+08 1.90830638e+08
 4.77064425e+08 4.54810042e+08 3.91065558e+08 2.88071106e+08
 5.77742797e+08 2.50166456e+08 2.75955635e+08 1.89657086e+08
 1.86054238e+08 4.59395360e+08 4.36101373e+08 3.70936126e+08
 2.77324140e+08 5.38285721e+08 2.66357817e+08 2.96548217e+08
 1.98156457e+08 1.94405862e+08 4.70449540e+08 4.45017343e+08
 3.73217566e+08 2.86191629e+08 5.29015000e+08]


In [5]:
rgb_total_mean = rgb_psum / pixel_count
rgb_total_var  = (rgb_psum_sq / pixel_count) - (rgb_total_mean ** 2)
rgb_total_std  = np.sqrt(rgb_total_var)

sh_total_mean = sh_psum / pixel_count
sh_total_var  = (sh_psum_sq / pixel_count) - (sh_total_mean ** 2)
sh_total_std  = np.sqrt(sh_total_var)

# output
print('RGB mean: '  + str(rgb_total_mean))
print('RGB std:  '  + str(rgb_total_std))

print('SH mean: '  + str(sh_total_mean))
print('SH std:  '  + str(sh_total_std))

RGB mean: [128.20173317 115.31897723 100.06612719]
RGB std:  [72.32526559 71.096137   70.01695119]
SH mean: [ 0.00824186  0.11539002  0.06089742  0.06155786  0.05825128  0.0593638
  0.09168289  0.05926862  0.11279657 -0.04009821  0.14060101  0.06198773
  0.0599228   0.05789624  0.05871703  0.08233076  0.0594893   0.09212159
 -0.08556332  0.1680364   0.06273864  0.0581194   0.05829075  0.05739062
  0.06550323  0.05976876  0.0617659 ]
SH std:  [0.36573967 0.36615546 0.31597748 0.31306109 0.50109065 0.48896791
 0.44744172 0.38749937 0.54356951 0.36309892 0.35698321 0.31197455
 0.3092862  0.49163777 0.47873379 0.43714261 0.37999565 0.52787927
 0.36710315 0.36049178 0.31901154 0.31674267 0.4975526  0.48383487
 0.44135966 0.38612645 0.52761967]


In [6]:
np.savez('../../Downloads/channel_mean_std.npz', rgb_mean=rgb_total_mean, rgb_std=rgb_total_std, sh_mean=sh_total_mean, sh_std=sh_total_std)

In [7]:
saved_array = np.load('../../Downloads/channel_mean_std.npz').items()
for item in saved_array:
    print(item)


('rgb_mean', array([128.20173317, 115.31897723, 100.06612719]))
('rgb_std', array([72.32526559, 71.096137  , 70.01695119]))
('sh_mean', array([ 0.00824186,  0.11539002,  0.06089742,  0.06155786,  0.05825128,
        0.0593638 ,  0.09168289,  0.05926862,  0.11279657, -0.04009821,
        0.14060101,  0.06198773,  0.0599228 ,  0.05789624,  0.05871703,
        0.08233076,  0.0594893 ,  0.09212159, -0.08556332,  0.1680364 ,
        0.06273864,  0.0581194 ,  0.05829075,  0.05739062,  0.06550323,
        0.05976876,  0.0617659 ]))
('sh_std', array([0.36573967, 0.36615546, 0.31597748, 0.31306109, 0.50109065,
       0.48896791, 0.44744172, 0.38749937, 0.54356951, 0.36309892,
       0.35698321, 0.31197455, 0.3092862 , 0.49163777, 0.47873379,
       0.43714261, 0.37999565, 0.52787927, 0.36710315, 0.36049178,
       0.31901154, 0.31674267, 0.4975526 , 0.48383487, 0.44135966,
       0.38612645, 0.52761967]))
